This notebook trains a logistic regression model, finds the optimal value of C, and reports F1 and log loss scores.

In [1]:

%matplotlib inline
import numpy as np
import pandas as pd
import zipfile
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [2]:
# Unzip data files into the "csv" subdirectory 
# (unless you have already done this since running the Data Set Up notebook)

# **IMPORTANT**  This will overwrite existing files in the "csv" folder in your local repo
# with the most recent data files from the data.zip file

# Unzip 80% training data
unzip_training_data = zipfile.ZipFile("data_subset.zip", "r")
unzip_training_data.extractall()
unzip_training_data.close()

# Unzip development and training data
unzip_test_data = zipfile.ZipFile("testing.zip", "r")
unzip_test_data.extractall()
unzip_test_data.close()

# Unzip full set of training data for creating predictions to submit to Kaggle
unzip_all_data = zipfile.ZipFile("data.zip", "r")
unzip_all_data.extractall()
unzip_all_data.close()

In [3]:
# Load these csv files into numpy arrays for testing on development data
train_data = np.loadtxt('csv/train_data.csv', delimiter=",")
train_labels = np.loadtxt('csv/train_labels.csv', dtype=str, delimiter=",")
dev_data = np.loadtxt('csv/dev_data.csv', delimiter=",")
dev_labels = np.loadtxt('csv/dev_labels.csv', dtype=str, delimiter=",")

KeyboardInterrupt: 

In [35]:
# Load these csv files into numpy arrays for creating predictions to submit to Kaggle
train_data_all = np.loadtxt('csv/train_data_all.csv', delimiter=",")
train_labels_all = np.loadtxt('csv/train_labels_all.csv', dtype=str, delimiter=",")
test_data_all = np.loadtxt('csv/test_data_all.csv', delimiter=",")

In [7]:
# print shapes to compare before and after csv conversion
print("train_data shape is", train_data.shape)
print("train_labels shape is", train_labels.shape)
print("dev_data shape is", dev_data.shape)
print("dev_labels shape is", dev_labels.shape)

train_data shape is (702439, 70)
train_labels shape is (702439,)
dev_data shape is (175610, 70)
dev_labels shape is (175610,)


In [8]:
print("train_data_all shape is", train_data_all.shape)
print("train_labels_all shape is", train_labels_all.shape)
print("test_data_all shape is", test_data_all.shape)

train_data_all shape is (878049, 70)
train_labels_all shape is (878049,)
test_data_all shape is (884262, 70)


In [5]:
# Set up functions for training logistic regression model and finding optimal value of C

def TrainLR(data, labels, test_data, C_value=1.0):
    """This function takes in training data and labels, testing data,
    and can accept different values of C (the learning rate).
    It trains a logistic regression model and returns the model and predicted probabilities.
    """
    LR = LogisticRegression(C=C_value, n_jobs = -1)
    LR.fit(data, labels)
    pp = LR.predict_proba(test_data)
    return LR, pp

def find_C(data, labels, dev_data, dev_labels, C_values):
    """Find optimal value of C in a logistic regression model.  
    
    Note that this cannot be used on test data from Kaggle 
    because we do not have labels for that data.  This function is intended to only be used
    in the development stage with the development data.
    """
    for C in C_values:      
        LR, pp = TrainLR(data, labels, dev_data, C, n_jobs = -1)
        predictions = LR.predict(dev_data)
        f1 = metrics.f1_score(dev_labels, predictions, average = "weighted")
        logloss = metrics.log_loss(dev_labels, pp)
        
        # Print F1 score and log loss for each value of k
        print("For C =", C, "the F1 score is", round(f1, 6), "and the Log Loss score is", round(logloss, 6))
    print("\n")

In [36]:
# IF there are additional changes to make to the data for this model
# that would be easier to do in pandas, uncomment and run this code. 
# This model works the same whether the data is in numpy or pandas, so presumably so do other models

#train_data = pd.DataFrame(train_data)
#train_labels = pd.DataFrame(train_labels)
#dev_data = pd.DataFrame(dev_data)
#dev_labels = pd.DataFrame(dev_labels)
train_data_all = pd.DataFrame(train_data_all, columns = get_feature_names)
#train_labels_all = pd.DataFrame(train_labels_all)
test_data_all = pd.DataFrame(test_data_all, columns = get_feature_names)

print(train_data_all.columns)
train_data_all.drop(['month_year'], axis = 1)
    
test_data_all.drop(['month_year'], axis = 1)


Index(['X', 'Y', 'hour', 'holidays', 'first_day', 'month_year', 'spring',
       'summer', 'fall', 'winter', 'PRCP', 'TMAX', 'TMIN', 'd_police',
       'rot_45_X', 'rot_45_Y', 'rot_30_X', 'rot_30_Y', 'rot_60_X', 'rot_60_Y',
       'radial_r', 'DayOfWeek_Friday', 'DayOfWeek_Monday',
       'DayOfWeek_Saturday', 'DayOfWeek_Sunday', 'DayOfWeek_Thursday',
       'DayOfWeek_Tuesday', 'DayOfWeek_Wednesday', 'PdDistrict_BAYVIEW',
       'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE', 'PdDistrict_MISSION',
       'PdDistrict_NORTHERN', 'PdDistrict_PARK', 'PdDistrict_RICHMOND',
       'PdDistrict_SOUTHERN', 'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007',
       'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012',
       'year_2013', 'year_2014', 'year_2015', 'dayparts_

,X,Y,hour,holidays,first_day,spring,summer,fall,winter,PRCP,...,year_2013,year_2014,year_2015,dayparts_early_afternoon,dayparts_early_evening,dayparts_early_morning,dayparts_late_afternoon,dayparts_late_evening,dayparts_late_morning,dayparts_late_night
0,-122.399588,37.735051,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
1,-122.391523,37.732432,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
2,-122.426002,37.792212,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
3,-122.437394,37.721412,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
4,-122.437394,37.721412,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
5,-122.459024,37.713172,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
6,-122.425616,37.739351,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
7,-122.412652,37.739750,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
8,-122.418700,37.765165,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14
9,-122.413935,37.798886,2.300000e+01,1.134103e-15,1.221126e-15,1.000000e+00,7.105523e-14,5.976261e-14,-2.581381e-14,7.976514e-14,...,3.143028e-13,-4.016814e-12,1.000000e+00,-4.851726e-14,4.313563e-14,4.774700e-14,2.782321e-14,1.000000e+00,5.322651e-14,1.176042e-14


In [34]:
print(test_data_all['month_year'])

0         1.480000e+02
1         1.480000e+02
2         1.480000e+02
3         1.480000e+02
4         1.480000e+02
5         1.480000e+02
6         1.480000e+02
7         1.480000e+02
8         1.480000e+02
9         1.480000e+02
10        1.480000e+02
11        1.480000e+02
12        1.480000e+02
13        1.480000e+02
14        1.480000e+02
15        1.480000e+02
16        1.480000e+02
17        1.480000e+02
18        1.480000e+02
19        1.480000e+02
20        1.480000e+02
21        1.480000e+02
22        1.480000e+02
23        1.480000e+02
24        1.480000e+02
25        1.480000e+02
26        1.480000e+02
27        1.480000e+02
28        1.480000e+02
29        1.480000e+02
              ...     
884232    8.690718e-13
884233    8.690718e-13
884234    8.690718e-13
884235    8.690718e-13
884236    8.690718e-13
884237    8.690718e-13
884238    8.690718e-13
884239    8.690718e-13
884240    8.690718e-13
884241    8.690718e-13
884242    8.690718e-13
884243    8.690718e-13
884244    8

In [ ]:
# Find the optimal value of C using the 80% training data and the development data
C_values = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 100.00, 1000.0]
find_C(train_data, train_labels, dev_data, dev_labels, C_values)

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


For C = 0.0001 the F1 score is 0.148899 and the Log Loss score is 3.011221
For C = 0.001 the F1 score is 0.152004 and the Log Loss score is 2.632073
For C = 0.01 the F1 score is 0.152955 and the Log Loss score is 2.544749
For C = 0.1 the F1 score is 0.153044 and the Log Loss score is 2.536512


In [10]:
# Train model with a single value of C with 80% training data and development data
C_value = 2.0
LR, pp = TrainLR(train_data, train_labels, dev_data, C_value)
logloss = metrics.log_loss(dev_labels, pp)
print(logloss)

2.55164421465


In [22]:
print(test_data_all[[13]])

[[ -1.22402131e+02   3.77993635e+01   2.30000000e+01   1.13410312e-15
    1.22112641e-15   1.48000000e+02   1.00000000e+00   7.10552343e-14
    5.97626091e-14  -2.58138065e-14   7.97651384e-14   5.70000000e+01
    4.90000000e+01   4.18974337e-03  -5.98141563e+01   1.13262456e+02
   -8.71005634e+01   9.39353141e+01  -2.84668165e+01   1.24899927e+02
    1.28105712e+02  -7.27129946e-15  -5.84756682e-15  -8.36377413e-14
    1.00000000e+00  -5.32049518e-14  -6.10362984e-14  -2.53076411e-14
   -3.28003920e-14   1.00000000e+00   9.51304769e-14   6.01701285e-14
    7.89733135e-14  -6.26487169e-14  -6.51588321e-15  -2.45310261e-13
    1.12259130e-13  -1.60885502e-14  -1.32793199e-13  -1.56762497e-13
    1.14828026e-13  -1.77027955e-14   1.00000000e+00  -1.17785832e-13
    1.87579739e-13   1.48970436e-13   1.13883935e-13   1.03036516e-13
   -1.78693525e-13  -1.49548207e-13  -5.10486444e-12   3.06833227e-12
   -1.05432682e-12   1.09916826e-12  -1.01266124e-15   2.79962986e-12
    2.19059370e-12  

In [47]:
logloss = metrics.log_loss(train_labels_all, LR.predict_proba(train_data_all))
print(logloss)

2.54909923503


In [37]:
# Before submitting to Kaggle, run the model on the full set of training data and test data
# using the optimal value for the model
C_value = 2.0
LR, pp = TrainLR(train_data_all, train_labels_all, test_data_all, C_value)

In [ ]:
clf = joblib.load('LR.pkl') 

In [38]:
from sklearn.externals import joblib
joblib.dump(LR, 'LR3.pkl') 

['LR3.pkl']

In [8]:
get_feature_names = ['X', 'Y', 'hour', 'holidays', 'first_day', 'month_year', 'spring',
       'summer', 'fall', 'winter', 'PRCP', 'TMAX', 'TMIN', 'd_police',
       'rot_45_X', 'rot_45_Y', 'rot_30_X', 'rot_30_Y', 'rot_60_X', 'rot_60_Y',
       'radial_r', 'DayOfWeek_Friday', 'DayOfWeek_Monday',
       'DayOfWeek_Saturday', 'DayOfWeek_Sunday', 'DayOfWeek_Thursday',
       'DayOfWeek_Tuesday', 'DayOfWeek_Wednesday', 'PdDistrict_BAYVIEW',
       'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE', 'PdDistrict_MISSION',
       'PdDistrict_NORTHERN', 'PdDistrict_PARK', 'PdDistrict_RICHMOND',
       'PdDistrict_SOUTHERN', 'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007',
       'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012',
       'year_2013', 'year_2014', 'year_2015', 'dayparts_early_afternoon',
       'dayparts_early_evening', 'dayparts_early_morning',
       'dayparts_late_afternoon', 'dayparts_late_evening',
       'dayparts_late_morning', 'dayparts_late_night']

headers = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE",
           "DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING",
           "FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL",
           "OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY",
           "SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE",
           "SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]

In [10]:
#Finds the top features (as a tuple)
topfeatures = [0]*len(headers)
for i in range(len(headers)):
    topfeatures[i] = sorted(enumerate(LR.coef_[i]), key=lambda tup: tup[1], reverse = True)[0:5]

#Extracts the indices of the top features
feature_index = [] 
for lst in topfeatures:
    for j in lst:
        feature_index.append(j[0])
feature_index.sort()

#Sets up the data for the table
feature_names = []
table_text = []
for i in feature_index:
    feature_names.append(get_feature_names[i])
    table_text.append([LR.coef_[j][i] for j in range(len(headers))])
    print(table_text)


In [12]:
print(feature_names)

['X', 'Y', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'hour', 'holidays', 'first_day', 'first_day', 'first_day', 'month_year', 'month_year', 'month_year', 'month_year', 'month_year', 'summer', 'summer', 'fall', 'fall', 'fall', 'winter', 'winter', 'TMAX', 'TMIN', 'd_police', 'd_police', 'd_police', 'd_police', 'd_police', 'd_police', 'd_police', 'd_police', 'd_police', 'rot_60_X', 'DayOfWeek_Sunday', 'DayOfWeek_Sunday', 'PdDistrict_BAYVIEW', 'PdDistrict_BAYVIEW', 'PdDistrict_BAYVIEW', 'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL', 'PdDistrict_CENTRAL', 'PdDistrict_CENTRAL', 'PdDistrict_CENTRAL', 'PdDistrict_CENTRAL', 'PdDistrict_CENTRAL', 'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE', 'PdDistrict_INGLESIDE', 'PdDistrict_INGLESIDE', 'PdDistrict_MISSION', 'PdDistrict_MISSION', 'PdDistrict_MISSION', 'PdDistrict_MISSION', 'PdDistrict_MISSION', 'PdDistrict_MISSION', 'PdDistrict_MISSION', 'PdDistrict_MISS

In [39]:
# Set up predictions for submission to Kaggle

data = pd.DataFrame(data=pp, 
                    index=[x for x in range(len(test_data_all))], 
                    columns=headers)
data.columns.name ="Id"
print(data.shape)
print(data)

(884262, 39)
Id             ARSON       ASSAULT    BAD CHECKS       BRIBERY  BURGLARY  \
0       4.081465e-11  9.908505e-10  3.179633e-52  1.391368e-07  0.095681   
1       4.093776e-11  9.921296e-10  3.182929e-52  1.394585e-07  0.095681   
2       3.918780e-11  1.056428e-09  3.432744e-52  1.299264e-07  0.095956   
3       3.357377e-11  9.923860e-10  3.518838e-52  1.409403e-07  0.095782   
4       3.357377e-11  9.923860e-10  3.518838e-52  1.409403e-07  0.095782   
5       3.432610e-11  9.961707e-10  3.963291e-52  1.213163e-07  0.095807   
6       3.357966e-11  9.887760e-10  3.540157e-52  1.401917e-07  0.095782   
7       3.361476e-11  9.897194e-10  3.548097e-52  1.403381e-07  0.095785   
8       3.250774e-11  1.023788e-09  3.253475e-52  1.628587e-07  0.095987   
9       3.883291e-11  1.071608e-09  3.671726e-52  1.250757e-07  0.095842   
10      3.358341e-11  9.882754e-10  3.555725e-52  1.399998e-07  0.095786   
11      3.266775e-11  1.025431e-09  3.256293e-52  1.634026e-07  0.095986   

Create zipped csv file for Kaggle
#### Update the filename first in all lines of the following code
Add something unique after our names to avoid overwriting other submission files

In [40]:
data.to_csv('Williams_Gascoigne_Vignola_Regression7.csv', index_label = "Id")

In [41]:
zip_probs = zipfile.ZipFile("Williams_Gascoigne_Vignola_Regression7.zip", "w")
zip_probs.write("Williams_Gascoigne_Vignola_Regression7.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_probs.close()

### Results from previous datasets and/or model parameters

**First Submission**   
Results on development data from dataset as of Saturday 11/18, with weather added, latitude outliers removed, binarized and normalized features:

For C = 0.0001 the F1 score is 0.147075 and the Log Loss score is 3.014795  
For C = 0.001 the F1 score is 0.150284 and the Log Loss score is 2.638404  
For C = 0.01 the F1 score is 0.151366 and the Log Loss score is 2.551881  
For C = 0.1 the F1 score is 0.151589 and the Log Loss score is 2.543797  
For C = 0.5 the F1 score is 0.151615 and the Log Loss score is 2.543427  
For C = 1.0 the F1 score is 0.151579 and the Log Loss score is 2.543396  
**For C = 2.0 the F1 score is 0.151605 and the Log Loss score is 2.543383**  
For C = 10.0 the F1 score is 0.151657 and the Log Loss score is 2.543385  
For C = 100.0 the F1 score is 0.151619 and the Log Loss score is 2.543447  
For C = 1000.0 the F1 score is 0.151616 and the Log Loss score is 2.543544  

Predictions on test data from training on full data set are in zip file ending with Regression1

Kaggle score from that zip file that we thought should have correlated with the above scores on dev data was 18.20988 (!?)  


**Second Submission**  
We changed our workflow along the way, so our first step was to re-run this notebook to confirm that we are unzipping and using the latest version of the data, in particular the full set of training and test data. 

Log loss on dev data after this step, with C=2.0 is 2.54338  
Predictions from this step are in zip file ending with Regression2  
Kaggle score is the same: 18.20989   
So the problem ended up being our test data was not normalized.


**Third Submission**  
On Cyprian's suggestion, we added the multi_class = 'multinomial' argument to the logistic regression model, because we don't have a binary output variable.  

Log loss on dev data after this step, with C=2.0 is 2.54267  
Predictions from this step are in zip file ending with Regression3  
Kaggle score on this set of predictions is:  33.37633   
This is even worse! Removed the multi_class argument from the model code.

**Fourth Submission**
Fixed a critical bug in submissions 2 and 3 that basically made them nonsense. Our test data was not normalized and now it is. Scored a 2.54 on Kaggle, not bad.
